In [ ]:
import numpy as np
from landlab import RasterModelGrid, HexModelGrid
from landlab.components import StreamPowerEroder, LinearDiffuser, FlowAccumulator
from landlab import imshow_grid
from matplotlib import pyplot as plt

In [ ]:
number_of_rows = 200            
number_of_columns = 200        
dxy = 100                        

In [ ]:
mg = RasterModelGrid((number_of_rows, number_of_columns), dxy)

# Create random noise
np.random.seed(0)                                       # seed set to zero so our figures are reproducible
mg_noise = np.random.rand(mg.number_of_nodes)/1000.     # intial noise on elevation grid

# set up the elevation on the grid
zr = mg.add_zeros('node', 'topographic__elevation')
zr += mg_noise

In [ ]:
East = 4
North = 4
West = 4
South = 1

In [ ]:
mg.set_status_at_node_on_edges(right=East, top=North, left=West, bottom=South)

In [ ]:
U = 1E-3

In [ ]:
uplift_rate = np.ones(mg.number_of_nodes)*U

## One-time uplift event
#fault_location = 10000  # [m] (original value = 4000)
#uplift_amount = 100 # [m] (original value = 10)
#zr[np.where(mg.node_y>fault_location)] += uplift_amount 

## Continuously-slipping fault, 
#fault_location = 10000  # [m] (Original value = 4000)
#low_uplift_rate = 1E-3 # [m/yr] (Original value = 0.0001)
#high_uplift_rate = 1E-2 # [m/yr] (Original value = 0.0004)
#uplift_rate[np.where(mg.node_y<fault_location)] = low_uplift_rate
#uplift_rate[np.where(mg.node_y>fault_location)] = high_uplift_rate

## Uplift Gradient
#low_uplift_rate = 0.0001 # [m/yr]
#high_uplift_rate = 0.004 # [m/yr]
#uplift_rate_gradient = (high_uplift_rate - low_uplift_rate)/(number_of_rows-3)
#uplift_rate = low_uplift_rate + ((mg.node_y / dxy)-1) * uplift_rate_gradient


In [ ]:
dt = 1000            # time step [yr]
tmax = 3E5        # time for the model loop to run [yr]

In [ ]:
total_time = 0 
t = np.arange(0, tmax, dt)

In [ ]:
min_drainage_area = 100000
K_hs = 1E-3
K_sp = 1E-4
m_sp = 0.5                  # exponent on drainage area in stream power equation 
n_sp = 1                    # exponent on slope in stream power equation

In [ ]:
frr = FlowAccumulator(mg) 
spr = StreamPowerEroder(mg, K_sp=K_sp, m_sp=m_sp, n_sp=n_sp, threshold_sp=0.0)
dfn = LinearDiffuser(mg, linear_diffusivity=K_hs, deposit = False)

In [ ]:
for ti in t:
    zr[mg.core_nodes] += uplift_rate[mg.core_nodes]*dt  # uplift the landscape
    dfn.run_one_step(dt)                                # diffuse the landscape
    frr.run_one_step()                                  # route flow
    spr.run_one_step(dt)                                # fluvial incision
    total_time += dt                                    # update time keeper
    print(total_time)

In [ ]:
plt.figure(1)
imshow_grid(mg, 'topographic__elevation', grid_units=('m', 'm'), cmap='terrain', allow_colorbar=True)
title_text = '$Year$='+str(total_time)  
plt.title(title_text)
plt.tight_layout()
plt.show()

In [ ]:
from landlab.components import (ChannelProfiler, ChiFinder, SteepnessFinder)

In [ ]:
prf = ChannelProfiler(mg, number_of_watersheds=1, main_channel_only=True, minimum_channel_threshold=min_drainage_area)
prf.run_one_step()

plt.figure(1)
prf.plot_profiles_in_map_view()
plt.title(title_text)
plt.tight_layout()
plt.show()

plt.figure(2)
prf.plot_profiles(xlabel='distance upstream (m)', ylabel='elevation (m)')
title_text = '$Year$='+str(total_time)  
plt.title(title_text)
plt.tight_layout()
plt.show()

plt.figure(3)
for i, outlet_id in enumerate(prf.data_structure):
    for j, segment_id in enumerate(prf.data_structure[outlet_id]):
        if j == 0:
            label = "channel {i}".format(i=i + 1)
        else:
            label = '_nolegend_'
        segment = prf.data_structure[outlet_id][segment_id]
        profile_ids = segment["ids"]
        color = segment["color"]
        plt.loglog(mg.at_node["drainage_area"][profile_ids], mg.at_node["topographic__steepest_slope"][profile_ids], '.', color=color, label=label)
plt.legend(loc="lower left")
plt.xlabel("drainage area (m^2)")
plt.ylabel("channel slope [m/m]")
title_text = '$Year$='+str(total_time)  
plt.title(title_text)
plt.tight_layout()
plt.show()

In [ ]:
sf = SteepnessFinder(mg, reference_concavity=m_sp/n_sp, min_drainage_area=min_drainage_area)
sf.calculate_steepnesses()

plt.figure(1)
prf.plot_profiles_in_map_view()
plt.title(title_text)
plt.tight_layout()
plt.show()

plt.figure(2)
for i, outlet_id in enumerate(prf.data_structure):
    for j, segment_id in enumerate(prf.data_structure[outlet_id]):
        if j == 0:
            label = "channel {i}".format(i=i + 1)
        else:
            label = '_nolegend_'
        segment = prf.data_structure[outlet_id][segment_id]
        profile_ids = segment["ids"]
        distance_upstream = segment["distances"]
        color = segment["color"]
        plt.plot(distance_upstream, mg.at_node["channel__steepness_index"][profile_ids], 'x', color=color, label=label)
plt.xlabel("distance upstream (m)")
plt.ylabel("steepness index")
plt.legend(loc="upper left")
title_text = '$Year$='+str(total_time)  
plt.title(title_text)
plt.tight_layout()
plt.show()

plt.figure(3)
imshow_grid(mg, "channel__steepness_index", grid_units=("m", "m"), var_name="Steepness index ", cmap="jet")
title_text = '$Year$='+str(total_time)  
plt.title(title_text)
plt.tight_layout()
plt.show()

In [ ]:
cf = ChiFinder(mg, reference_concavity=m_sp/n_sp, min_drainage_area=min_drainage_area)
cf.calculate_chi()

plt.figure(1)
prf.plot_profiles_in_map_view()
plt.title(title_text)
plt.tight_layout()
plt.show()

plt.figure(2)
for i, outlet_id in enumerate(prf.data_structure):
    for j, segment_id in enumerate(prf.data_structure[outlet_id]):
        if j == 0:
            label = "channel {i}".format(i=i + 1)
        else:
            label = '_nolegend_'
        segment = prf.data_structure[outlet_id][segment_id]
        profile_ids = segment["ids"]
        color = segment["color"]
        plt.plot(mg.at_node["channel__chi_index"][profile_ids], mg.at_node["topographic__elevation"][profile_ids], color=color, label=label)
plt.xlabel("chi index (m)")
plt.ylabel("elevation (m)")
plt.legend(loc="lower right")
title_text = '$Year$='+str(total_time)  
plt.title(title_text)
plt.tight_layout()
plt.show()

plt.figure(3)
imshow_grid(mg, "channel__chi_index", grid_units=("m", "m"), var_name="Chi index (m)", cmap="jet")
title_text = '$Year$='+str(total_time)  
plt.title(title_text)
plt.tight_layout()
plt.show()

In [ ]:
zr[mg.core_nodes] += 50
dt = 1000            
tmax = 1E4  
t = np.arange(0, tmax, dt)
for ti in t:
    zr[mg.core_nodes] += uplift_rate[mg.core_nodes]*dt  
    dfn.run_one_step(dt)                                
    frr.run_one_step()                                  
    spr.run_one_step(dt)                                
    total_time += dt                                    
    print(total_time)
plt.figure(1)
imshow_grid(mg, 'topographic__elevation', grid_units=('m', 'm'), cmap='terrain', allow_colorbar=True)
title_text = '$Year$='+str(total_time)  
plt.title(title_text)
plt.tight_layout()
plt.show()

In [ ]:
prf.run_one_step()

plt.figure(1)
prf.plot_profiles_in_map_view()
plt.title(title_text)
plt.tight_layout()
plt.show()

plt.figure(2)
prf.plot_profiles(xlabel='distance upstream (m)', ylabel='elevation (m)')
title_text = '$Year$='+str(total_time)  
plt.title(title_text)
plt.tight_layout()
plt.show()

plt.figure(3)
for i, outlet_id in enumerate(prf.data_structure):
    for j, segment_id in enumerate(prf.data_structure[outlet_id]):
        if j == 0:
            label = "channel {i}".format(i=i + 1)
        else:
            label = '_nolegend_'
        segment = prf.data_structure[outlet_id][segment_id]
        profile_ids = segment["ids"]
        color = segment["color"]
        plt.loglog(mg.at_node["drainage_area"][profile_ids], mg.at_node["topographic__steepest_slope"][profile_ids], '.', color=color, label=label)
plt.legend(loc="lower left")
plt.xlabel("drainage area (m^2)")
plt.ylabel("channel slope [m/m]")
title_text = '$Year$='+str(total_time)  
plt.title(title_text)
plt.tight_layout()
plt.show()

sf.calculate_steepnesses()

plt.figure(4)
for i, outlet_id in enumerate(prf.data_structure):
    for j, segment_id in enumerate(prf.data_structure[outlet_id]):
        if j == 0:
            label = "channel {i}".format(i=i + 1)
        else:
            label = '_nolegend_'
        segment = prf.data_structure[outlet_id][segment_id]
        profile_ids = segment["ids"]
        distance_upstream = segment["distances"]
        color = segment["color"]
        plt.plot(distance_upstream, mg.at_node["channel__steepness_index"][profile_ids], 'x', color=color, label=label)
plt.xlabel("distance upstream (m)")
plt.ylabel("steepness index")
plt.legend(loc="upper left")
title_text = '$Year$='+str(total_time)  
plt.title(title_text)
plt.tight_layout()
plt.show()

plt.figure(5)
imshow_grid(mg, "channel__steepness_index", grid_units=("m", "m"), var_name="Steepness index ", cmap="jet")
title_text = '$Year$='+str(total_time)  
plt.title(title_text)
plt.tight_layout()
plt.show()

cf.calculate_chi()

plt.figure(6)
for i, outlet_id in enumerate(prf.data_structure):
    for j, segment_id in enumerate(prf.data_structure[outlet_id]):
        if j == 0:
            label = "channel {i}".format(i=i + 1)
        else:
            label = '_nolegend_'
        segment = prf.data_structure[outlet_id][segment_id]
        profile_ids = segment["ids"]
        color = segment["color"]
        plt.plot(mg.at_node["channel__chi_index"][profile_ids], mg.at_node["topographic__elevation"][profile_ids], color=color, label=label)
plt.xlabel("chi index (m)")
plt.ylabel("elevation (m)")
plt.legend(loc="lower right")
title_text = '$Year$='+str(total_time)  
plt.title(title_text)
plt.tight_layout()
plt.show()

plt.figure(7)
imshow_grid(mg, "channel__chi_index", grid_units=("m", "m"), var_name="Chi index (m)", cmap="jet")
title_text = '$Year$='+str(total_time)  
plt.title(title_text)
plt.tight_layout()
plt.show()

In [ ]:
dt = 1000            
tmax = 4E4  
t = np.arange(0, tmax, dt)
for ti in t:
    zr[mg.core_nodes] += uplift_rate[mg.core_nodes]*dt  
    dfn.run_one_step(dt)                                
    frr.run_one_step()                                  
    spr.run_one_step(dt)                                
    total_time += dt                                    
    print(total_time)
plt.figure(1)
imshow_grid(mg, 'topographic__elevation', grid_units=('m', 'm'), cmap='terrain', allow_colorbar=True)
title_text = '$Year$='+str(total_time)  
plt.title(title_text)
plt.tight_layout()
plt.show()

In [ ]:
prf.run_one_step()

plt.figure(1)
prf.plot_profiles_in_map_view()
plt.title(title_text)
plt.tight_layout()
plt.show()

plt.figure(2)
prf.plot_profiles(xlabel='distance upstream (m)', ylabel='elevation (m)')
title_text = '$Year$='+str(total_time)  
plt.title(title_text)
plt.tight_layout()
plt.show()

plt.figure(3)
for i, outlet_id in enumerate(prf.data_structure):
    for j, segment_id in enumerate(prf.data_structure[outlet_id]):
        if j == 0:
            label = "channel {i}".format(i=i + 1)
        else:
            label = '_nolegend_'
        segment = prf.data_structure[outlet_id][segment_id]
        profile_ids = segment["ids"]
        color = segment["color"]
        plt.loglog(mg.at_node["drainage_area"][profile_ids], mg.at_node["topographic__steepest_slope"][profile_ids], '.', color=color, label=label)
plt.legend(loc="lower left")
plt.xlabel("drainage area (m^2)")
plt.ylabel("channel slope [m/m]")
title_text = '$Year$='+str(total_time)  
plt.title(title_text)
plt.tight_layout()
plt.show()

sf.calculate_steepnesses()

plt.figure(4)
for i, outlet_id in enumerate(prf.data_structure):
    for j, segment_id in enumerate(prf.data_structure[outlet_id]):
        if j == 0:
            label = "channel {i}".format(i=i + 1)
        else:
            label = '_nolegend_'
        segment = prf.data_structure[outlet_id][segment_id]
        profile_ids = segment["ids"]
        distance_upstream = segment["distances"]
        color = segment["color"]
        plt.plot(distance_upstream, mg.at_node["channel__steepness_index"][profile_ids], 'x', color=color, label=label)
plt.xlabel("distance upstream (m)")
plt.ylabel("steepness index")
plt.legend(loc="upper left")
title_text = '$Year$='+str(total_time)  
plt.title(title_text)
plt.tight_layout()
plt.show()

plt.figure(5)
imshow_grid(mg, "channel__steepness_index", grid_units=("m", "m"), var_name="Steepness index ", cmap="jet")
title_text = '$Year$='+str(total_time)  
plt.title(title_text)
plt.tight_layout()
plt.show()

cf.calculate_chi()

plt.figure(6)
for i, outlet_id in enumerate(prf.data_structure):
    for j, segment_id in enumerate(prf.data_structure[outlet_id]):
        if j == 0:
            label = "channel {i}".format(i=i + 1)
        else:
            label = '_nolegend_'
        segment = prf.data_structure[outlet_id][segment_id]
        profile_ids = segment["ids"]
        color = segment["color"]
        plt.plot(mg.at_node["channel__chi_index"][profile_ids], mg.at_node["topographic__elevation"][profile_ids], color=color, label=label)
plt.xlabel("chi index (m)")
plt.ylabel("elevation (m)")
plt.legend(loc="lower right")
title_text = '$Year$='+str(total_time)  
plt.title(title_text)
plt.tight_layout()
plt.show()

plt.figure(7)
imshow_grid(mg, "channel__chi_index", grid_units=("m", "m"), var_name="Chi index (m)", cmap="jet")
title_text = '$Year$='+str(total_time)  
plt.title(title_text)
plt.tight_layout()
plt.show()

In [ ]:
dt = 1000            
tmax = 1.5E5  
t = np.arange(0, tmax, dt)
for ti in t:
    zr[mg.core_nodes] += uplift_rate[mg.core_nodes]*dt  
    dfn.run_one_step(dt)                                
    frr.run_one_step()                                  
    spr.run_one_step(dt)                                
    total_time += dt                                    
    print(total_time)
plt.figure(1)
imshow_grid(mg, 'topographic__elevation', grid_units=('m', 'm'), cmap='terrain', allow_colorbar=True)
title_text = '$Year$='+str(total_time)  
plt.title(title_text)
plt.tight_layout()
plt.show()

In [ ]:
uplift_rate[np.where(mg.node_y>10000)] = U*2
dt = 1000            
tmax = 1E5  
t = np.arange(0, tmax, dt)
for ti in t:
    zr[mg.core_nodes] += uplift_rate[mg.core_nodes]*dt  
    dfn.run_one_step(dt)                                
    frr.run_one_step()                                  
    spr.run_one_step(dt)                                
    total_time += dt                                    
    print(total_time)
plt.figure(1)
imshow_grid(mg, 'topographic__elevation', grid_units=('m', 'm'), cmap='terrain', allow_colorbar=True)
title_text = '$Year$='+str(total_time)  
plt.title(title_text)
plt.tight_layout()
plt.show()

In [ ]:
prf.run_one_step()

plt.figure(1)
prf.plot_profiles_in_map_view()
plt.title(title_text)
plt.tight_layout()
plt.show()

plt.figure(2)
prf.plot_profiles(xlabel='distance upstream (m)', ylabel='elevation (m)')
title_text = '$Year$='+str(total_time)  
plt.title(title_text)
plt.tight_layout()
plt.show()

plt.figure(3)
for i, outlet_id in enumerate(prf.data_structure):
    for j, segment_id in enumerate(prf.data_structure[outlet_id]):
        if j == 0:
            label = "channel {i}".format(i=i + 1)
        else:
            label = '_nolegend_'
        segment = prf.data_structure[outlet_id][segment_id]
        profile_ids = segment["ids"]
        color = segment["color"]
        plt.loglog(mg.at_node["drainage_area"][profile_ids], mg.at_node["topographic__steepest_slope"][profile_ids], '.', color=color, label=label)
plt.legend(loc="lower left")
plt.xlabel("drainage area (m^2)")
plt.ylabel("channel slope [m/m]")
title_text = '$Year$='+str(total_time)  
plt.title(title_text)
plt.tight_layout()
plt.show()

sf.calculate_steepnesses()

plt.figure(4)
for i, outlet_id in enumerate(prf.data_structure):
    for j, segment_id in enumerate(prf.data_structure[outlet_id]):
        if j == 0:
            label = "channel {i}".format(i=i + 1)
        else:
            label = '_nolegend_'
        segment = prf.data_structure[outlet_id][segment_id]
        profile_ids = segment["ids"]
        distance_upstream = segment["distances"]
        color = segment["color"]
        plt.plot(distance_upstream, mg.at_node["channel__steepness_index"][profile_ids], 'x', color=color, label=label)
plt.xlabel("distance upstream (m)")
plt.ylabel("steepness index")
plt.legend(loc="upper left")
title_text = '$Year$='+str(total_time)  
plt.title(title_text)
plt.tight_layout()
plt.show()

plt.figure(5)
imshow_grid(mg, "channel__steepness_index", grid_units=("m", "m"), var_name="Steepness index ", cmap="jet")
title_text = '$Year$='+str(total_time)  
plt.title(title_text)
plt.tight_layout()
plt.show()

cf.calculate_chi()

plt.figure(6)
for i, outlet_id in enumerate(prf.data_structure):
    for j, segment_id in enumerate(prf.data_structure[outlet_id]):
        if j == 0:
            label = "channel {i}".format(i=i + 1)
        else:
            label = '_nolegend_'
        segment = prf.data_structure[outlet_id][segment_id]
        profile_ids = segment["ids"]
        color = segment["color"]
        plt.plot(mg.at_node["channel__chi_index"][profile_ids], mg.at_node["topographic__elevation"][profile_ids], color=color, label=label)
plt.xlabel("chi index (m)")
plt.ylabel("elevation (m)")
plt.legend(loc="lower right")
title_text = '$Year$='+str(total_time)  
plt.title(title_text)
plt.tight_layout()
plt.show()

plt.figure(7)
imshow_grid(mg, "channel__chi_index", grid_units=("m", "m"), var_name="Chi index (m)", cmap="jet")
title_text = '$Year$='+str(total_time)  
plt.title(title_text)
plt.tight_layout()
plt.show()